In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
 
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import warnings
warnings.filterwarnings('ignore')
%config Completer.use_jedi = False 

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import re
from wordcloud import STOPWORDS
from collections import defaultdict
#%%time
from tqdm._tqdm_notebook import tqdm_notebook
tqdm_notebook.pandas()
from transformers import AutoTokenizer,TFBertModel

max_len = 36

import tensorflow as tf
tf.config.experimental.list_physical_devices('GPU')

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow.keras.losses import CategoricalCrossentropy,BinaryCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy,BinaryAccuracy
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.utils import plot_model
from tensorflow.keras.layers import Input, Dense

from nltk.corpus import stopwords
from wordcloud import WordCloud

In [ ]:
train_df = pd.read_csv("../input/nlp-getting-started/train.csv")
test_df = pd.read_csv("../input/nlp-getting-started/test.csv")

In [ ]:
train_df.head()

In [ ]:
test_df.head()

# Text preprocessing

In [ ]:
train_df.text=[a.lower() for a in train_df.text]
test_df.text=[a.lower() for a in test_df.text]
train_df.head()

In [ ]:
train_df['word_count'] = train_df['text'].apply(lambda x: len(str(x).split()))
test_df['word_count'] = test_df['text'].apply(lambda x: len(str(x).split()))
train_df['unique_word_count'] = train_df['text'].apply(lambda x: len(set(str(x).split())))
test_df['unique_word_count'] = test_df['text'].apply(lambda x: len(set(str(x).split())))
train_df['url_count'] = train_df['text'].apply(lambda x: len([w for w in str(x).lower().split() if 'http' in w or 'https' in w]))
test_df['url_count'] = test_df['text'].apply(lambda x: len([w for w in str(x).lower().split() if 'http' in w or 'https' in w]))
train_df['mean_word_length'] = train_df['text'].apply(lambda x: np.mean([len(w) for w in str(x).split()]))
test_df['mean_word_length'] = test_df['text'].apply(lambda x: np.mean([len(w) for w in str(x).split()]))
train_df['char_count'] = train_df['text'].apply(lambda x: len(str(x)))
test_df['char_count'] = test_df['text'].apply(lambda x: len(str(x)))


In [ ]:
import re
def remove_URL(text):
    url = re.compile(r'https?://\S+|www\.\S+')
    return url.sub(r'',text)
train_df['text'] = train_df['text'].apply(lambda x : remove_URL(x))
test_df['text'] = test_df['text'].apply(lambda x : remove_URL(x))

In [ ]:
def remove_html(text):
    html=re.compile(r'<.*?>')
    return html.sub(r'',text)
train_df['text'] = train_df['text'].apply(lambda x : remove_html(x))
test_df['text'] = test_df['text'].apply(lambda x : remove_html(x))

In [ ]:
def remove_emoji(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)
train_df['text'] = train_df['text'].apply(lambda x: remove_emoji(x))
test_df['text'] = test_df['text'].apply(lambda x: remove_emoji(x))

In [ ]:
def cleaner(tweet):
    tweet = re.sub(r"Typhoon-Devastated", "typhoon devastated", tweet)
    tweet = re.sub(r"TyphoonDevastated", "typhoon devastated", tweet)
    tweet = re.sub(r"typhoondevastated", "typhoon devastated", tweet)
    tweet = re.sub(r"MH370", "Malaysia Airlines Flight", tweet)
    tweet = re.sub(r"MH", "Malaysia Airlines Flight", tweet)
    tweet = re.sub(r"mh370", "Malaysia Airlines Flight", tweet)
    tweet = re.sub(r"year-old", "years old", tweet)
    tweet = re.sub(r"yearold", "years old", tweet)
    tweet = re.sub(r"yr old", "years old", tweet)
    tweet = re.sub(r"PKK", "Kurdistan Workers Party", tweet)
    tweet = re.sub(r"MP", "madhya pradesh", tweet)
    tweet = re.sub(r"rly", "railway", tweet)
    tweet = re.sub(r"CDT", "Central Daylight Time", tweet)
    tweet = re.sub(r"sensorsenso", "sensor senso", tweet)
    tweet = re.sub(r"pm", "", tweet)
    tweet = re.sub(r"PM", "", tweet)
    tweet = re.sub(r"nan", " ", tweet)
    tweet = re.sub(r"terrorismturn", "terrorism turn", tweet)
    tweet = re.sub(r"epicente", "epicenter", tweet)
    tweet = re.sub(r"epicenterr", "epicenter", tweet)
    tweet = re.sub(r"WAwildfire", "Washington Wildfire", tweet)
    tweet = re.sub(r"prebreak", "pre break", tweet)
    tweet = re.sub(r"nowplaying", "now playing", tweet)
    tweet = re.sub(r"RT", "retweet", tweet)
    tweet = re.sub(r"EbolaOutbreak", "Ebola Outbreak", tweet)
    tweet = re.sub(r"LondonFire", "London Fire", tweet)
    tweet = re.sub(r"IDFire", "Idaho Fire", tweet)
    tweet = re.sub(r"withBioterrorism&use", "with Bioterrorism & use", tweet)
    tweet = re.sub(r"NASAHurricane", "NASA Hurricane", tweet)
    tweet = re.sub(r"withweapons", "with weapons", tweet)
    tweet = re.sub(r"NuclearPower", "Nuclear Power", tweet)
    tweet = re.sub(r"WhiteTerrorism", "White Terrorism", tweet)
    tweet = re.sub(r"MyanmarFlood", "Myanmar Flood", tweet)
    tweet = re.sub(r"ExtremeWeather", "Extreme Weather", tweet)
    tweet = re.sub(r"%20", " ", tweet)
    tweet = re.sub(r"%", " ", tweet)
    tweet = re.sub(r"@", " ", tweet)
    tweet = re.sub(r"'", " ", tweet)
    tweet = re.sub(r"\x89û_", " ", tweet)
    tweet = re.sub(r"\x89ûò", " ", tweet)
    tweet = re.sub(r"16yr", "16 year", tweet)
    tweet = re.sub(r"re\x89û_", " ", tweet)
    tweet = re.sub(r"\x89û", " ", tweet)
    tweet = re.sub(r"\x89Û", " ", tweet)
    tweet = re.sub(r"re\x89Û", "re ", tweet)
    tweet = re.sub(r"re\x89û", "re ", tweet)
    tweet = re.sub(r"\x89ûª", "'", tweet)
    tweet = re.sub(r"\x89û", " ", tweet)
    tweet = re.sub(r"\x89ûò", " ", tweet)
    tweet = re.sub(r"\x89Û_", "", tweet)
    tweet = re.sub(r"\x89ÛÒ", "", tweet)
    tweet = re.sub(r"\x89ÛÓ", "", tweet)
    tweet = re.sub(r"\x89ÛÏWhen", "When", tweet)
    tweet = re.sub(r"\x89ÛÏ", "", tweet)
    tweet = re.sub(r"China\x89Ûªs", "China's", tweet)
    tweet = re.sub(r"let\x89Ûªs", "let's", tweet)
    tweet = re.sub(r"\x89Û÷", "", tweet)
    tweet = re.sub(r"\x89Ûª", "", tweet)
    tweet = re.sub(r"\x89Û\x9d", "", tweet)
    tweet = re.sub(r"å_", "", tweet)
    tweet = re.sub(r"\x89Û¢", "", tweet)
    tweet = re.sub(r"\x89Û¢åÊ", "", tweet)
    tweet = re.sub(r"fromåÊwounds", "from wounds", tweet)
    tweet = re.sub(r"åÊ", "", tweet)
    tweet = re.sub(r"åÈ", "", tweet)
    tweet = re.sub(r"JapÌ_n", "Japan", tweet)    
    tweet = re.sub(r"Ì©", "e", tweet)
    tweet = re.sub(r"å¨", "", tweet)
    tweet = re.sub(r"SuruÌ¤", "Suruc", tweet)
    tweet = re.sub(r"åÇ", "", tweet)
    tweet = re.sub(r"å£3million", "3 million", tweet)
    tweet = re.sub(r"åÀ", "", tweet)
    tweet = re.sub(r"he's", "he is", tweet)
    tweet = re.sub(r"there's", "there is", tweet)
    tweet = re.sub(r"We're", "We are", tweet)
    tweet = re.sub(r"That's", "That is", tweet)
    tweet = re.sub(r"won't", "will not", tweet)
    tweet = re.sub(r"they're", "they are", tweet)
    tweet = re.sub(r"Can't", "Cannot", tweet)
    tweet = re.sub(r"wasn't", "was not", tweet)
    tweet = re.sub(r"don\x89Ûªt", "do not", tweet)
    tweet = re.sub(r"aren't", "are not", tweet)
    tweet = re.sub(r"isn't", "is not", tweet)
    tweet = re.sub(r"What's", "What is", tweet)
    tweet = re.sub(r"haven't", "have not", tweet)
    tweet = re.sub(r"hasn't", "has not", tweet)
    tweet = re.sub(r"There's", "There is", tweet)
    tweet = re.sub(r"He's", "He is", tweet)
    tweet = re.sub(r"It's", "It is", tweet)
    tweet = re.sub(r"You're", "You are", tweet)
    tweet = re.sub(r"I'M", "I am", tweet)
    tweet = re.sub(r"Im", "I am", tweet)
    tweet = re.sub(r"shouldn't", "should not", tweet)
    tweet = re.sub(r"wouldn't", "would not", tweet)
    tweet = re.sub(r"i'm", "I am", tweet)
    tweet = re.sub(r"I\x89Ûªm", "I am", tweet)
    tweet = re.sub(r"I'm", "I am", tweet)
    tweet = re.sub(r"Isn't", "is not", tweet)
    tweet = re.sub(r"Here's", "Here is", tweet)
    tweet = re.sub(r"you've", "you have", tweet)
    tweet = re.sub(r"you\x89Ûªve", "you have", tweet)
    tweet = re.sub(r"we're", "we are", tweet)
    tweet = re.sub(r"what's", "what is", tweet)
    tweet = re.sub(r"couldn't", "could not", tweet)
    tweet = re.sub(r"we've", "we have", tweet)
    tweet = re.sub(r"it\x89Ûªs", "it is", tweet)
    tweet = re.sub(r"doesn\x89Ûªt", "does not", tweet)
    tweet = re.sub(r"It\x89Ûªs", "It is", tweet)
    tweet = re.sub(r"Here\x89Ûªs", "Here is", tweet)
    tweet = re.sub(r"who's", "who is", tweet)
    tweet = re.sub(r"I\x89Ûªve", "I have", tweet)
    tweet = re.sub(r"y'all", "you all", tweet)
    tweet = re.sub(r"can\x89Ûªt", "cannot", tweet)
    tweet = re.sub(r"would've", "would have", tweet)
    tweet = re.sub(r"it'll", "it will", tweet)
    tweet = re.sub(r"we'll", "we will", tweet)
    tweet = re.sub(r"wouldn\x89Ûªt", "would not", tweet)
    tweet = re.sub(r"We've", "We have", tweet)
    tweet = re.sub(r"he'll", "he will", tweet)
    tweet = re.sub(r"Y'all", "You all", tweet)
    tweet = re.sub(r"Weren't", "Were not", tweet)
    tweet = re.sub(r"Didn't", "Did not", tweet)
    tweet = re.sub(r"they'll", "they will", tweet)
    tweet = re.sub(r"they'd", "they would", tweet)
    tweet = re.sub(r"DON'T", "DO NOT", tweet)
    tweet = re.sub(r"That\x89Ûªs", "That is", tweet)
    tweet = re.sub(r"they've", "they have", tweet)
    tweet = re.sub(r"i'd", "I would", tweet)
    tweet = re.sub(r"should've", "should have", tweet)
    tweet = re.sub(r"You\x89Ûªre", "You are", tweet)
    tweet = re.sub(r"where's", "where is", tweet)
    tweet = re.sub(r"Don\x89Ûªt", "Do not", tweet)
    tweet = re.sub(r"we'd", "we would", tweet)
    tweet = re.sub(r"i'll", "I will", tweet)
    tweet = re.sub(r"weren't", "were not", tweet)
    tweet = re.sub(r"They're", "They are", tweet)
    tweet = re.sub(r"Can\x89Ûªt", "Cannot", tweet)
    tweet = re.sub(r"you\x89Ûªll", "you will", tweet)
    tweet = re.sub(r"I\x89Ûªd", "I would", tweet)
    tweet = re.sub(r"let's", "let us", tweet)
    tweet = re.sub(r"it's", "it is", tweet)
    tweet = re.sub(r"can't", "can not", tweet)
    tweet = re.sub(r"cant", "can not", tweet)
    tweet = re.sub(r"don't", "do not", tweet)
    tweet = re.sub(r"dont", "do not", tweet)
    tweet = re.sub(r"you're", "you are", tweet)
    tweet = re.sub(r"i've", "I have", tweet)
    tweet = re.sub(r"that's", "that is", tweet)
    tweet = re.sub(r"i'll", "I will", tweet)
    tweet = re.sub(r"doesn't", "does not", tweet)
    tweet = re.sub(r"i'd", "I would", tweet)
    tweet = re.sub(r"didn't", "did not", tweet)
    tweet = re.sub(r"ain't", "am not", tweet)
    tweet = re.sub(r"you'll", "you will", tweet)
    tweet = re.sub(r"I've", "I have", tweet)
    tweet = re.sub(r"Don't", "do not", tweet)
    tweet = re.sub(r"I'll", "I will", tweet)
    tweet = re.sub(r"I'd", "I would", tweet)
    tweet = re.sub(r"Let's", "Let us", tweet)
    tweet = re.sub(r"you'd", "You would", tweet)
    tweet = re.sub(r"It's", "It is", tweet)
    tweet = re.sub(r"Ain't", "am not", tweet)
    tweet = re.sub(r"Haven't", "Have not", tweet)
    tweet = re.sub(r"Could've", "Could have", tweet)
    tweet = re.sub(r"youve", "you have", tweet)  
    tweet = re.sub(r"donå«t", "do not", tweet)
    return tweet
train_df['text'] = train_df['text'].apply(lambda s : cleaner(s))
test_df['text'] = test_df['text'].apply(lambda s : cleaner(s))

In [ ]:
train_df['text'] = train_df['text'].str.replace('   ', ' ')
train_df['text'] = train_df['text'].str.replace('     ', ' ')
train_df['text'] = train_df['text'].str.replace('\xa0 \xa0 \xa0', ' ')
train_df['text'] = train_df['text'].str.replace('  ', ' ')
train_df['text'] = train_df['text'].str.replace('—', ' ')
train_df['text'] = train_df['text'].str.replace('–', ' ')
test_df['text'] = test_df['text'].str.replace('   ', ' ')
test_df['text'] = test_df['text'].str.replace('     ', ' ')
test_df['text'] = test_df['text'].str.replace('\xa0 \xa0 \xa0', ' ')
test_df['text'] = test_df['text'].str.replace('  ', ' ')
test_df['text'] = test_df['text'].str.replace('—', ' ')
test_df['text'] = test_df['text'].str.replace('–', ' ')

# Hashtags can say a lot about situation
# I calculate all hashtags and mean target by every hashtag

In [ ]:
train_df['hashtags']=train_df['text']
for i in range(train_df.shape[0]):
    j=0
    a=[]
    s=train_df.text[i]
    while(j<len(s)):
        if(s[j]=='#'):
            j+=1
            j1=j
            while(j<len(s) and s[j]!=' '):
                j+=1
            a.append(s[j1:j])
        else:
            j+=1
    train_df.hashtags[i]=a

In [ ]:
train_df.head()

In [ ]:
test_df['hashtags']=test_df['text']
for i in range(test_df.shape[0]):
    j=0
    a=[]
    s=test_df.text[i]
    while(j<len(s)):
        if(s[j]=='#'):
            j+=1
            j1=j
            while(j<len(s) and s[j]!=' ' and s[j]!='\n' and s[j]!='#'):
                j+=1
            a.append(s[j1:j])
        else:
            j+=1
    test_df.hashtags[i]=a
test_df.head()

In [ ]:
st1=set()
for i in train_df.hashtags:
    for j in i:
        st1.add(j)
st2=set()
for i in test_df.hashtags:
    for j in i:
        st2.add(j)
print(st2-st1)                

In [ ]:
train_df.head()

In [ ]:
m={}
for i in range(train_df.shape[0]):
    for j in train_df.hashtags[i]:
        if(j not in m):
            m[j]=train_df['target'][i]
        else:
            m[j]+=train_df['target'][i]

In [ ]:
x=[]
for i in m.keys():
    x.append((m[i],i))
x=sorted(x)
x=list(reversed(x))
print(x[:40])

In [ ]:
train_df['mean_hashtags_target']=train_df['target']
for i in range(train_df.shape[0]):
    s=0
    for j in train_df.hashtags[i]:
        if(j not in m):
            s+=0
        else:
            s+=m[j]
    if(len(train_df.hashtags[i])==0):
        s=0
    else:
        s/=len(train_df.hashtags[i])
    train_df['mean_hashtags_target'][i]=s

In [ ]:
test_df['mean_hashtags_target']=test_df['text']
for i in range(test_df.shape[0]):
    s=0
    for j in test_df.hashtags[i]:
        if(j not in m):
            s+=0
        else:
            s+=m[j]
    if(len(test_df.hashtags[i])==0):
        s=0
    else:
        s/=len(test_df.hashtags[i])
    test_df['mean_hashtags_target'][i]=s

# Null preprocessing

In [ ]:
train_df['location']=train_df['location'].fillna('NAN')
test_df['location']=test_df['location'].fillna('NAN')
train_df['keyword']=train_df['keyword'].fillna('NAN')
test_df['keyword']=test_df['keyword'].fillna('NAN')

In [ ]:
train_df['mean_target_keyword']=train_df['target']
train_df['mean_target_keyword']=train_df['keyword'].map(train_df.groupby('keyword')['target'].mean())

# Mean target by keyword

In [ ]:
test_df['mean_target_keyword']=test_df['keyword'].map(train_df.groupby('keyword')['target'].mean())

# Count !,?,hashtags

In [ ]:
train_df['hashtags_count']=[a.count('#') for a in train_df.text]
test_df['hashtags_count']=[a.count('#') for a in test_df.text]
train_df['!_count']=[a.count('!') for a in train_df.text]
test_df['!_count']=[a.count('!') for a in test_df.text]
train_df['?_count']=[a.count('?') for a in train_df.text]
test_df['?_count']=[a.count('?') for a in test_df.text]
train_df['?_count'].unique()

# Install bert model

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('bert-large-uncased')
bert = TFBertModel.from_pretrained('bert-large-uncased')  

In [ ]:
x_train=tokenizer(
    text=train_df.text.tolist(),
    add_special_tokens=True,
    max_length=36,
    truncation=True,
    padding=True, 
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = True,
    verbose = True)

In [ ]:
print(x_train['attention_mask'])

In [ ]:
input_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_ids")
input_mask = Input(shape=(max_len,), dtype=tf.int32, name="attention_mask")
embeddings = bert(input_ids,attention_mask = input_mask)[1]
out = tf.keras.layers.Dropout(0.1)(embeddings)
out = Dense(128, activation='relu')(out)
out = tf.keras.layers.Dropout(0.1)(out)
out = Dense(32,activation = 'relu')(out)
y = Dense(1,activation = 'sigmoid')(out)
model = tf.keras.Model(inputs=[input_ids, input_mask], outputs=y)
model.layers[2].trainable = True

In [ ]:
model.summary()

In [ ]:
optimizer = Adam(
    learning_rate=6e-06, # this learning rate is for bert model.
    epsilon=1e-08,
    decay=0.01,
    clipnorm=1.0)
loss = BinaryCrossentropy(from_logits = True)
metric = BinaryAccuracy('accuracy'),
model.compile(
    optimizer = optimizer,
    loss = loss, 
    metrics = metric)

In [ ]:
plot_model(model, show_shapes = True)

In [ ]:
y_train=train_df['target'].values
final = model.fit(
    x ={'input_ids':x_train['input_ids'],'attention_mask':x_train['attention_mask']} ,
    y = y_train,
    epochs=9,
    batch_size=8  
)

In [ ]:
x_test = tokenizer(
    text=test_df.text.tolist(),
    add_special_tokens=True,
    max_length=36,
    truncation=True,
    padding=True, 
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = True,
    verbose = True)
data=pd.DataFrame()
predicted = model.predict({'input_ids':x_test['input_ids'],'attention_mask':x_test['attention_mask']})
y_predicted = np.where(predicted>0.5,1,0)
y_predicted = y_predicted.reshape((1,3263))[0]
data['id'] = test_df.id
data['target'] = y_predicted

In [ ]:
data.to_csv('submission.csv',index = False)
data.head()

# I get info about BERT model from https://www.kaggle.com/aishwarya2210/prediction-of-tweets-using-bert-model Thank!
# You also can copy my notebook and fit gyperparams to improve score

# Unfortunately I cant use my feature and I only feat my model on text
# Maybe you try make DataFrame with BERT prediction+ text features and fit xgboost or lighbgm on this DataFrame